In [1]:
import pandas as pd #importing pandas library
Writer= pd.ExcelWriter('Nation With Namo.xlsx', engine='xlsxwriter') #to save data into multiple tab in single excel

In [2]:
df=pd.read_csv(r'C:\Users\Akhil\Downloads\Bihar_Election_Results .csv',sep=',',index_col=None) #reading the csv file
df.head()

,Election Year,Position,Name,Votes,Votes %,Party,AC name,AC No
0,2010,1,Rajesh Singh,42289,29.40%,Janata Dal (United),Valmiki Nagar,1
1,2010,2,Mukesh Kumar Kushwaha,27618,19.20%,Rashtriya Janata Dal,Valmiki Nagar,1
2,2010,3,Dheerendra Pratap Singh,20886,14.50%,Bahujan Samaj Party,Valmiki Nagar,1
3,2010,4,Irshad Hussain,17747,12.40%,Indian National Congress,Valmiki Nagar,1
4,2010,5,Deep Narayan Mahato,14047,9.80%,Independent,Valmiki Nagar,1


In [3]:
data=df.copy()  #making shallow copy of data
data.tail()     #taking last 5 rows

,Election Year,Position,Name,Votes,Votes %,Party,AC name,AC No
12296,2005-Feb,15,Chandeshwar Prasad Gupt,534,0.60%,Independent,Chakai,243
12297,2005-Feb,16,Tirpit Singh,498,0.60%,Independent,Chakai,243
12298,2005-Feb,17,Surendra Rajbansi,437,0.50%,Independent,Chakai,243
12299,2005-Feb,18,Nawin Kumar Paswan,415,0.50%,Independent,Chakai,243
12300,2005-Feb,19,Ajay Kumar,407,0.50%,Independent,Chakai,243


In [4]:
data['Election Year'].unique()  # unique data present in Election year column

array(['2010', '2015', '2005-Oct', '2005-Feb'], dtype=object)

## Assumption :- 
I am mapping  distinct element in Election year as ;<br>
'2005-Feb':Election 1 <br>
'2005-Oct':Election 2 <br>
'2010':Election 3<br>
'2015':Election 4


In [5]:
#mapping Election year with my convenient
data['Election Year']=data['Election Year'].map({'2005-Feb':'Election 1','2005-Oct':'Election 2','2010':'Election 3','2015':'Election 4'})
data.head()

,Election Year,Position,Name,Votes,Votes %,Party,AC name,AC No
0,Election 3,1,Rajesh Singh,42289,29.40%,Janata Dal (United),Valmiki Nagar,1
1,Election 3,2,Mukesh Kumar Kushwaha,27618,19.20%,Rashtriya Janata Dal,Valmiki Nagar,1
2,Election 3,3,Dheerendra Pratap Singh,20886,14.50%,Bahujan Samaj Party,Valmiki Nagar,1
3,Election 3,4,Irshad Hussain,17747,12.40%,Indian National Congress,Valmiki Nagar,1
4,Election 3,5,Deep Narayan Mahato,14047,9.80%,Independent,Valmiki Nagar,1


In [6]:
data.shape  #checking the number of rows and column in datatset

(12301, 8)

## 1.> Determine the winning party for every election at the state level. A party which has the highest total votes is considered the winner here.


In [7]:
## Winner by constitiuency in every election(list of all the winner with constituency number)
winner_list_df=pd.pivot_table(data,values=['Votes'],index=['Election Year','AC No'],aggfunc=max) #Each Ac No with maximum seat
winner_list_df=winner_list_df.reset_index() #resting from multiindexing to normal indexing column
winner_list_df.info() # columns nformation
winner_list_df.head() #printing top 5 rows

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 966 entries, 0 to 965
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Election Year  966 non-null    object
 1   AC No          966 non-null    int64 
 2   Votes          966 non-null    int64 
dtypes: int64(2), object(1)
memory usage: 22.8+ KB


,Election Year,AC No,Votes
0,Election 1,1,19389
1,Election 1,2,59151
2,Election 1,3,20961
3,Election 1,4,30421
4,Election 1,5,26161


In [8]:
##creating primary key to connet winner list with original data source to more relevant information
winner_list_df['Primary Key']=winner_list_df['AC No'].map(str)+'_'+winner_list_df['Votes'].map(str) #conctructing primary key to join two df
data['Primary Key']=data['AC No'].map(str)+'_'+data['Votes'].map(str)

In [9]:
## left merge of two df and dropping the mismatched key elements
winner_list=data.merge(winner_list_df,how='left',left_on='Primary Key',right_on='Primary Key')
winner_list=winner_list.dropna()
winner_list=winner_list.drop(['AC No_y','Election Year_y','Votes_y'],axis=1) #dropping redundant column after merging
winner_list=winner_list.rename(columns={'Votes_x':'Votes','Election Year_x':'Election Year','Ac No_x':'AC No'}) #renaming the column
winner_list.reset_index(drop=True,inplace=True)
winner_list.shape  #printing the shape of dataframe(to get idea about number of rows and column in df)

(966, 9)

In [10]:
winner_list.head() #printing top 5 rows

,Election Year,Position,Name,Votes,Votes %,Party,AC name,AC No_x,Primary Key
0,Election 3,1,Rajesh Singh,42289,29.40%,Janata Dal (United),Valmiki Nagar,1,1_42289
1,Election 3,1,Bhagirathi Devi,51993,41.50%,Bharatiya Janta Party,Ramnagar,2,2_51993
2,Election 3,1,Satish Chandra Dubey,45022,38.10%,Bharatiya Janta Party,Narkatiaganj,3,3_45022
3,Election 3,1,Prabhat Ranjan Singh,67510,50.40%,Janata Dal (United),Bagaha,4,4_67510
4,Election 3,1,Vinay Bihari,38381,33.40%,Independent,Lauriya,5,5_38381


In [11]:
winner_list_grouped=winner_list.groupby(['Election Year','Party']).agg({'Position':'count'})
winner_list_grouped=winner_list_grouped.reset_index()  #converting multiindex into normal index
winner_list_grouped['Primary_key']=winner_list_grouped['Position'].map(str)+'_'+winner_list_grouped['Election Year'].map(str)
##winner_list_grouped['Primary_key_temp']=winner_list_grouped['Election Year']+winner_list_grouped['Votes'].map(str)+'_'+winner_list_grouped['AC No'].map(str)+'_'+winner_list_grouped['Name']
winner_list_grouped

,Election Year,Party,Position,Primary_key
0,Election 1,Bahujan Samaj Party,2,2_Election 1
1,Election 1,Bharatiya Janta Party,37,37_Election 1
2,Election 1,Communist Party Of India,3,3_Election 1
3,Election 1,Communist Party Of India (MARXIST),1,1_Election 1
4,Election 1,Communist Party Of India (MARXIST-LENINIST) (L...,7,7_Election 1
5,Election 1,Independent,15,15_Election 1
6,Election 1,Indian National Congress,10,10_Election 1
7,Election 1,Janata Dal (United),55,55_Election 1
8,Election 1,Lok Jan Shakti Party,29,29_Election 1
9,Election 1,Nationalist Congress Party,3,3_Election 1


In [12]:
result_grouped=winner_list_grouped.groupby(['Election Year'])
result_grouped=result_grouped.max()
result_grouped=result_grouped.reset_index()
result_grouped['Primary_key']=result_grouped['Position'].map(str)+'_'+result_grouped['Election Year'].map(str)
result_grouped.head()

,Election Year,Party,Position,Primary_key
0,Election 1,Samajwadi Party,75,75_Election 1
1,Election 2,Samajwadi Party,88,88_Election 2
2,Election 3,Rashtriya Janata Dal,113,113_Election 3
3,Election 4,Rashtriya Lok Samta Party,80,80_Election 4


In [13]:
temp_res_grouped=result_grouped.drop(['Party'],axis=1)
winner_every_election=temp_res_grouped.merge(winner_list_grouped,how='inner',on='Primary_key') #merging the column to get party name
winner_every_election=winner_every_election.dropna()  #dropping non matched rows
winner_every_election=winner_every_election.drop(['Position_x','Election Year_y','Primary_key'],axis=1) #dropping Redundant column
winner_every_election=winner_every_election.rename(columns={'Election Year_x':'Election Year','Position_y':'Seat Count'}) #renaming the column

#### Final Result

In [14]:
winner_every_election   #final winner in each election

,Election Year,Party,Seat Count
0,Election 1,Rashtriya Janata Dal,75
1,Election 2,Janata Dal (United),88
2,Election 3,Janata Dal (United),113
3,Election 4,Rashtriya Janata Dal,80


In [15]:
##saving file to excel
winner_every_election.to_excel(Writer,sheet_name='Winning Party Every Election',index=False)

## 2.> determine the list of swing seats for the state. A swing seat is a constituency which has a different winner in every election. The elections are held in the order 2005-Feb, 2005-Oct, 2010 and 2015.

In [16]:
winner_list.head()  #all the winner in all four election

,Election Year,Position,Name,Votes,Votes %,Party,AC name,AC No_x,Primary Key
0,Election 3,1,Rajesh Singh,42289,29.40%,Janata Dal (United),Valmiki Nagar,1,1_42289
1,Election 3,1,Bhagirathi Devi,51993,41.50%,Bharatiya Janta Party,Ramnagar,2,2_51993
2,Election 3,1,Satish Chandra Dubey,45022,38.10%,Bharatiya Janta Party,Narkatiaganj,3,3_45022
3,Election 3,1,Prabhat Ranjan Singh,67510,50.40%,Janata Dal (United),Bagaha,4,4_67510
4,Election 3,1,Vinay Bihari,38381,33.40%,Independent,Lauriya,5,5_38381


In [17]:
seat_temp=winner_list.pivot(index='AC No_x',columns='Election Year',values='Party').reset_index()
seat_temp

Election Year,AC No_x,Election 1,Election 2,Election 3,Election 4
0,1,Rashtriya Janata Dal,Rashtriya Janata Dal,Janata Dal (United),Independent
1,2,Janata Dal (United),Janata Dal (United),Bharatiya Janta Party,Bharatiya Janta Party
2,3,Bharatiya Janta Party,Bharatiya Janta Party,Bharatiya Janta Party,Indian National Congress
3,4,Nationalist Congress Party,Bharatiya Janta Party,Janata Dal (United),Bharatiya Janta Party
4,5,Samajwadi Party,Indian National Congress,Independent,Bharatiya Janta Party
...,...,...,...,...,...
238,239,Independent,Independent,Janata Dal (United),Bharatiya Janta Party
239,240,Rashtriya Janata Dal,Bharatiya Janta Party,Janata Dal (United),Indian National Congress
240,241,Independent,Independent,Janata Dal (United),Rashtriya Janata Dal
241,242,Lok Jan Shakti Party,Independent,Janata Dal (United),Bharatiya Janta Party


In [18]:
def swing_seat(data):
    if (data[-1] ==data[-2]):
        return 'Seat Match Between two differenct election'
    elif (data[-1] ==data[-3]):
        return 'Seat Match Between two differenct election'
    elif (data[-1] ==data[-4]):
        return 'Seat Match Between two differenct election'
    elif (data[-2] ==data[-3]):
        return 'Seat Match Between two differenct election'
    elif (data[-2] ==data[-4]):
        return 'Seat Match Between two differenct election'
    if (data[-3] ==data[-4]):
        return 'Seat Match Between two differenct election'
    else:
        return 'Swing Seat'
seat_temp['is_swing']=seat_temp.apply(swing_seat,axis=1)
swing_seat_final=seat_temp.dropna(axis=0).reset_index(drop=True)

In [19]:
swing_seat_final[swing_seat_final['is_swing']=='Swing Seat'].to_excel(Writer,sheet_name='Swing_seat',index=False)

## 3.> Determine the top 8 parties at the state-level in each election in terms of getting the most votes.


In [20]:
top_parties=winner_list_grouped.groupby('Election Year').apply(lambda x:x.sort_values(['Position'],ascending=False)).reset_index(drop=True)
top_parties=top_parties.drop('Primary_key',axis=1)
top_8_parties=top_parties.groupby('Election Year').head(8)
top_8_parties=top_8_parties.rename(columns={'Position':'Seat Count'})
top_8_parties

,Election Year,Party,Seat Count
0,Election 1,Rashtriya Janata Dal,75
1,Election 1,Janata Dal (United),55
2,Election 1,Bharatiya Janta Party,37
3,Election 1,Lok Jan Shakti Party,29
4,Election 1,Independent,15
5,Election 1,Indian National Congress,10
6,Election 1,Communist Party Of India (MARXIST-LENINIST) (L...,7
7,Election 1,Samajwadi Party,4
12,Election 2,Janata Dal (United),88
13,Election 2,Bharatiya Janta Party,55


In [21]:
top_8_parties.to_excel(Writer,sheet_name='top8 parties every election',index=False)

## 4>. Determine the minimum votes that each party in the top 8 of the 2015 election secured among all the past elections. Do this for all constituencies.


In [22]:
top8_2015_election=top_8_parties[top_8_parties['Election Year']=='Election 4'].reset_index(drop=True)
top8_2015_election # #top 8 parties in 2015 election

,Election Year,Party,Seat Count
0,Election 4,Rashtriya Janata Dal,80
1,Election 4,Janata Dal (United),71
2,Election 4,Bharatiya Janta Party,51
3,Election 4,Indian National Congress,27
4,Election 4,Independent,4
5,Election 4,Communist Party Of India (Marxist-Leninist) (L...,3
6,Election 4,Lok Jan Shakti Party,2
7,Election 4,Rashtriya Lok Samta Party,2


In [23]:
mandatory_list=top8_2015_election.Party.to_list()
mandatory_list

['Rashtriya Janata Dal',
 'Janata Dal (United)',
 'Bharatiya Janta Party',
 'Indian National Congress',
 'Independent',
 'Communist Party Of India (Marxist-Leninist) (Liberation)',
 'Lok Jan Shakti Party',
 'Rashtriya Lok Samta Party']

In [24]:
#merging these top 8 parties with original data go get votes
data_for_min_votes=data.merge(top8_2015_election,how='left',left_on='Party',right_on='Party')
data_for_min_votes=data_for_min_votes.dropna().reset_index(drop=True)
data_for_min_votes=data_for_min_votes.drop(['Primary Key','Election Year_y','Seat Count'],axis=1)
data_for_min_votes=data_for_min_votes.rename(columns={'Election Year_x':'Election Year'})
data_for_min_votes.head()

,Election Year,Position,Name,Votes,Votes %,Party,AC name,AC No
0,Election 3,1,Rajesh Singh,42289,29.40%,Janata Dal (United),Valmiki Nagar,1
1,Election 3,2,Mukesh Kumar Kushwaha,27618,19.20%,Rashtriya Janata Dal,Valmiki Nagar,1
2,Election 3,4,Irshad Hussain,17747,12.40%,Indian National Congress,Valmiki Nagar,1
3,Election 3,5,Deep Narayan Mahato,14047,9.80%,Independent,Valmiki Nagar,1
4,Election 3,7,Saket Kumar Pathak,4428,3.10%,Independent,Valmiki Nagar,1


In [25]:
min_votes_temp=data_for_min_votes.groupby(['Election Year','AC No']).agg({'Votes':'min'}).reset_index() 
min_votes_temp#minimum votes from each constituency fr top 8 parties

,Election Year,AC No,Votes
0,Election 1,1,1067
1,Election 1,2,1098
2,Election 1,3,402
3,Election 1,4,1298
4,Election 1,5,440
...,...,...,...
961,Election 4,239,2822
962,Election 4,240,3372
963,Election 4,241,516
964,Election 4,242,880


In [26]:
data.head()

,Election Year,Position,Name,Votes,Votes %,Party,AC name,AC No,Primary Key
0,Election 3,1,Rajesh Singh,42289,29.40%,Janata Dal (United),Valmiki Nagar,1,1_42289
1,Election 3,2,Mukesh Kumar Kushwaha,27618,19.20%,Rashtriya Janata Dal,Valmiki Nagar,1,1_27618
2,Election 3,3,Dheerendra Pratap Singh,20886,14.50%,Bahujan Samaj Party,Valmiki Nagar,1,1_20886
3,Election 3,4,Irshad Hussain,17747,12.40%,Indian National Congress,Valmiki Nagar,1,1_17747
4,Election 3,5,Deep Narayan Mahato,14047,9.80%,Independent,Valmiki Nagar,1,1_14047


In [27]:
##constructing primary key to connect it with original data
min_votes_temp['Primary Key min']=min_votes_temp['Election Year']+'_'+min_votes_temp['Votes'].map(str)+'_'+min_votes_temp['AC No'].map(str)
data['Primary Key min']=data['Election Year']+'_'+data['Votes'].map(str)+'_'+data['AC No'].map(str)
min_votes=data.merge(min_votes_temp,how='left',left_on='Primary Key min',right_on='Primary Key min')
min_votes=min_votes.dropna().reset_index(drop=True)
min_votes=min_votes.drop(['Election Year_y','Votes_y','AC No_y','Primary Key'],axis=1)
min_votes=min_votes.rename(columns={'Election Year_x':'Election Year','Votes_x':'Votes','AC No_y':'AC No'})
min_votes

,Election Year,Position,Name,Votes,Votes %,Party,AC name,AC No_x,Primary Key min
0,Election 3,14,Pahawari Yadaw,743,0.50%,Independent,Valmiki Nagar,1,Election 3_743_1
1,Election 3,16,Nathu Ravi,900,0.70%,Independent,Ramnagar,2,Election 3_900_2
2,Election 3,12,Nandlalram,817,0.70%,Independent,Narkatiaganj,3,Election 3_817_3
3,Election 3,14,Abhishekh Nath Tiwari,722,0.50%,Independent,Bagaha,4,Election 3_722_4
4,Election 3,6,Sajjad Ansari,2850,2.50%,Independent,Lauriya,5,Election 3_2850_5
...,...,...,...,...,...,...,...,...,...
965,Election 1,21,Ram Balak Prasad,140,0.10%,Independent,Warsaliganj,239,Election 1_140_239
966,Election 1,12,Binod Chaudhary,644,0.70%,Independent,Sikandra (SC),240,Election 1_644_240
967,Election 1,15,Prakampan Kumar Bhaskar,318,0.30%,Independent,Jamui,241,Election 1_318_241
968,Election 1,14,Arjun Prasad,186,0.20%,Independent,Jhajha,242,Election 1_186_242


In [28]:
##it might be possible two different party will have minmum votes and out of which one cannot be from top 8 party from 
##2015 election so all those party which will not exist in top 8 list will return false
def min_votes_party(data):
    if data in mandatory_list:
        return 'True'
    else:
        return 'False'
min_votes['Valid Party']=min_votes['Party'].apply(min_votes_party)

In [29]:
min_votes_temp=min_votes[min_votes['Valid Party']!='False'] #removing non top 8 party of 2015 election(Electon 4)
min_votes_final=min_votes_temp.drop(['Primary Key min','Valid Party'],axis=1)
min_votes_final=min_votes_final.rename(columns={'AC No_x':'AC No'})

#### Final Result

In [30]:
min_votes_final.head()

,Election Year,Position,Name,Votes,Votes %,Party,AC name,AC No
0,Election 3,14,Pahawari Yadaw,743,0.50%,Independent,Valmiki Nagar,1
1,Election 3,16,Nathu Ravi,900,0.70%,Independent,Ramnagar,2
2,Election 3,12,Nandlalram,817,0.70%,Independent,Narkatiaganj,3
3,Election 3,14,Abhishekh Nath Tiwari,722,0.50%,Independent,Bagaha,4
4,Election 3,6,Sajjad Ansari,2850,2.50%,Independent,Lauriya,5


In [31]:
min_votes_final.to_excel(Writer,sheet_name='min votes top8 every election',index=False)

## 5.> Determine the consistently tri-polar constituencies. I.e Constituencies in which 3 parties or more got greater than 10% of the vote shares in each election. The parties can be different for different elections.

In [32]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12301 entries, 0 to 12300
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Election Year    12301 non-null  object
 1   Position         12301 non-null  int64 
 2   Name             12301 non-null  object
 3   Votes            12301 non-null  int64 
 4   Votes %          12301 non-null  object
 5   Party            12301 non-null  object
 6   AC name          12301 non-null  object
 7   AC No            12301 non-null  int64 
 8   Primary Key      12301 non-null  object
 9   Primary Key min  12301 non-null  object
dtypes: int64(3), object(7)
memory usage: 961.1+ KB


In [33]:
df1=data['Votes %'].str.split('%',expand=True) ## renmoving the percentage sign
df1.head()

,0,1
0,29.40,
1,19.20,
2,14.50,
3,12.40,
4,9.80,


In [34]:
data_ops=pd.concat([data,df1],axis=1)
data_ops.head()

,Election Year,Position,Name,Votes,Votes %,Party,AC name,AC No,Primary Key,Primary Key min,0,1
0,Election 3,1,Rajesh Singh,42289,29.40%,Janata Dal (United),Valmiki Nagar,1,1_42289,Election 3_42289_1,29.40,
1,Election 3,2,Mukesh Kumar Kushwaha,27618,19.20%,Rashtriya Janata Dal,Valmiki Nagar,1,1_27618,Election 3_27618_1,19.20,
2,Election 3,3,Dheerendra Pratap Singh,20886,14.50%,Bahujan Samaj Party,Valmiki Nagar,1,1_20886,Election 3_20886_1,14.50,
3,Election 3,4,Irshad Hussain,17747,12.40%,Indian National Congress,Valmiki Nagar,1,1_17747,Election 3_17747_1,12.40,
4,Election 3,5,Deep Narayan Mahato,14047,9.80%,Independent,Valmiki Nagar,1,1_14047,Election 3_14047_1,9.80,


In [35]:
data_ops=data_ops.drop(['Votes %',1],axis=1)  #removing redundant column
data_ops=data_ops.rename(columns={0:'Votes %'}) # renamng the column with valid name
data_ops['Votes %']=data_ops['Votes %'].map(float) #converting object to float
data_ops.head() #printing top 5 values

,Election Year,Position,Name,Votes,Party,AC name,AC No,Primary Key,Primary Key min,Votes %
0,Election 3,1,Rajesh Singh,42289,Janata Dal (United),Valmiki Nagar,1,1_42289,Election 3_42289_1,29.4
1,Election 3,2,Mukesh Kumar Kushwaha,27618,Rashtriya Janata Dal,Valmiki Nagar,1,1_27618,Election 3_27618_1,19.2
2,Election 3,3,Dheerendra Pratap Singh,20886,Bahujan Samaj Party,Valmiki Nagar,1,1_20886,Election 3_20886_1,14.5
3,Election 3,4,Irshad Hussain,17747,Indian National Congress,Valmiki Nagar,1,1_17747,Election 3_17747_1,12.4
4,Election 3,5,Deep Narayan Mahato,14047,Independent,Valmiki Nagar,1,1_14047,Election 3_14047_1,9.8


In [36]:
def greater_than_ten_per(data):
    if data>10.00:
        return "True"
    else:
        return "False"
data_ops['istrue']=data_ops['Votes %'].apply(greater_than_ten_per)
tripolar=data_ops[data_ops['istrue']!='False']

In [37]:
tripolar_grouped=tripolar.groupby(['Election Year','AC No']).agg({'Party':'count'}).reset_index()
def data_morethaneq_3(data):
    if data>=3:
        return 'True'
    else:
        return 'False'
tripolar_grouped['is_correct']=tripolar_grouped['Party'].apply(data_morethaneq_3)
tripolar_grouped=tripolar_grouped[tripolar_grouped['is_correct']!='False']
tripolar_grouped.to_excel('1.xlsx',index=False)
tripolar_constituency=tripolar_grouped.drop('is_correct',axis=1)
tripolar_constituency=tripolar_constituency.rename(columns={'Party':'count of more than 10% vote'})

#### Final Result

In [38]:
tripolar_constituency.to_excel(Writer,sheet_name='tripolar constituency',index=False)

## 6.> Determine the stronghold party-seats. A stronghold seat is one that is won by the same party in all the elections.

In [39]:
winner_list.head()

,Election Year,Position,Name,Votes,Votes %,Party,AC name,AC No_x,Primary Key
0,Election 3,1,Rajesh Singh,42289,29.40%,Janata Dal (United),Valmiki Nagar,1,1_42289
1,Election 3,1,Bhagirathi Devi,51993,41.50%,Bharatiya Janta Party,Ramnagar,2,2_51993
2,Election 3,1,Satish Chandra Dubey,45022,38.10%,Bharatiya Janta Party,Narkatiaganj,3,3_45022
3,Election 3,1,Prabhat Ranjan Singh,67510,50.40%,Janata Dal (United),Bagaha,4,4_67510
4,Election 3,1,Vinay Bihari,38381,33.40%,Independent,Lauriya,5,5_38381


In [40]:
temp_sh=winner_list.pivot(index='AC No_x',columns='Election Year',values='Party').reset_index()
temp_sh.head()

Election Year,AC No_x,Election 1,Election 2,Election 3,Election 4
0,1,Rashtriya Janata Dal,Rashtriya Janata Dal,Janata Dal (United),Independent
1,2,Janata Dal (United),Janata Dal (United),Bharatiya Janta Party,Bharatiya Janta Party
2,3,Bharatiya Janta Party,Bharatiya Janta Party,Bharatiya Janta Party,Indian National Congress
3,4,Nationalist Congress Party,Bharatiya Janta Party,Janata Dal (United),Bharatiya Janta Party
4,5,Samajwadi Party,Indian National Congress,Independent,Bharatiya Janta Party


In [41]:
def stronghold(data):
    if (data[-1]==data[-2]) and (data[-1]==data[-3]) and (data[-1]==data[-4]):
        return 'Stronghold'
    else:
        return 'Not Stronghold'
temp_sh['is_stronghold']=temp_sh.apply(stronghold,axis=1)
stronghold_party=temp_sh.reset_index(drop=True)
stronghold_party[stronghold_party['is_stronghold']=='Stronghold']

Election Year,AC No_x,Election 1,Election 2,Election 3,Election 4,is_stronghold
6,7,Bharatiya Janta Party,Bharatiya Janta Party,Bharatiya Janta Party,Bharatiya Janta Party,Stronghold
7,8,Bharatiya Janta Party,Bharatiya Janta Party,Bharatiya Janta Party,Bharatiya Janta Party,Stronghold
9,10,Bharatiya Janta Party,Bharatiya Janta Party,Bharatiya Janta Party,Bharatiya Janta Party,Stronghold
24,25,Bharatiya Janta Party,Bharatiya Janta Party,Bharatiya Janta Party,Bharatiya Janta Party,Stronghold
40,41,Janata Dal (United),Janata Dal (United),Janata Dal (United),Janata Dal (United),Stronghold
71,72,Janata Dal (United),Janata Dal (United),Janata Dal (United),Janata Dal (United),Stronghold
104,105,Bharatiya Janta Party,Bharatiya Janta Party,Bharatiya Janta Party,Bharatiya Janta Party,Stronghold
111,112,Janata Dal (United),Janata Dal (United),Janata Dal (United),Janata Dal (United),Stronghold
131,132,Janata Dal (United),Janata Dal (United),Janata Dal (United),Janata Dal (United),Stronghold
148,149,Janata Dal (United),Janata Dal (United),Janata Dal (United),Janata Dal (United),Stronghold


In [42]:
stronghold_party[stronghold_party['is_stronghold']=='Not Stronghold'].to_excel(Writer,sheet_name='stronghold party',index=False)

##  7.> Determine close contest constituencies. Close contest seats are those in which the top 2 finishers had a difference of less than 10% votes. Find seats in which there was a close contest in all the elections. If there is no such seat, then relax the criteria to 15%.


In [43]:
data_ops=data_ops.drop(['Primary Key','Primary Key min','istrue'],axis=1) #removing redundant column
data_ops.head()

,Election Year,Position,Name,Votes,Party,AC name,AC No,Votes %
0,Election 3,1,Rajesh Singh,42289,Janata Dal (United),Valmiki Nagar,1,29.4
1,Election 3,2,Mukesh Kumar Kushwaha,27618,Rashtriya Janata Dal,Valmiki Nagar,1,19.2
2,Election 3,3,Dheerendra Pratap Singh,20886,Bahujan Samaj Party,Valmiki Nagar,1,14.5
3,Election 3,4,Irshad Hussain,17747,Indian National Congress,Valmiki Nagar,1,12.4
4,Election 3,5,Deep Narayan Mahato,14047,Independent,Valmiki Nagar,1,9.8


In [44]:
temp_top_2=data_ops.groupby(['Election Year','AC No']).apply(lambda x:x.sort_values(['Votes %'],ascending=False)).reset_index(drop=True)
top2=temp_top_2.groupby(['Election Year','AC No']).head(2)
top2['Election Year'].unique()
top2

,Election Year,Position,Name,Votes,Party,AC name,AC No,Votes %
0,Election 1,1,Rajesh Singh,19389,Rashtriya Janata Dal,Valmiki Nagar,1,30.6
1,Election 1,2,Ramadhar Yadav,13287,Samajwadi Party,Valmiki Nagar,1,21.0
11,Election 1,1,Purnamasi Ram,59151,Janata Dal (United),Ramnagar (SC),2,49.0
12,Election 1,2,Naresh Ram,27481,Rashtriya Janata Dal,Ramnagar (SC),2,22.8
23,Election 1,1,Chandra Mohan Rai,20961,Bharatiya Janta Party,Narkatiaganj,3,21.5
...,...,...,...,...,...,...,...,...
12260,Election 4,2,Ajoy Pratap,58328,Bharatiya Janta Party,Jamui,241,37.0
12275,Election 4,1,Rabindra Yadav,65537,Bharatiya Janta Party,Jhajha,242,40.0
12276,Election 4,2,Damodar Rawat,43451,Janata Dal (United),Jhajha,242,26.6
12289,Election 4,1,Savitri Devi,47064,Rashtriya Janata Dal,Chakai,243,31.3


In [45]:
##applying lag function
top2['lead_Votes%']=top2.groupby(['Election Year','AC No'])['Votes %'].shift(-1) #if AC No changes it will give Nan
top2['per_diff']=top2.loc[:,'Votes %']-top2.loc[:,'lead_Votes%']
top2['Election Year'].unique()

<ipython-input-45-16f53d5f2530>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  top2['lead_Votes%']=top2.groupby(['Election Year','AC No'])['Votes %'].shift(-1) #if AC No changes it will give Nan
<ipython-input-45-16f53d5f2530>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  top2['per_diff']=top2.loc[:,'Votes %']-top2.loc[:,'lead_Votes%']


array(['Election 1', 'Election 2', 'Election 3', 'Election 4'],
      dtype=object)

In [46]:
def less_than_10_per(data):
    if data < 10:
        return 'True'
    else:
        return 'False'
top2['isclose_contest']=top2['per_diff'].apply(less_than_10_per)
top2=top2.drop(['lead_Votes%'],axis=1)
closed_contest_constituency=top2[top2['isclose_contest']!='False']

<ipython-input-46-ab56a92f2135>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  top2['isclose_contest']=top2['per_diff'].apply(less_than_10_per)


In [47]:
analysis=closed_contest_constituency
analysis.head()

,Election Year,Position,Name,Votes,Party,AC name,AC No,Votes %,per_diff,isclose_contest
0,Election 1,1,Rajesh Singh,19389,Rashtriya Janata Dal,Valmiki Nagar,1,30.6,9.6,True
23,Election 1,1,Chandra Mohan Rai,20961,Bharatiya Janta Party,Narkatiaganj,3,21.5,0.3,True
59,Election 1,1,Dilip Verma,26161,Samajwadi Party,Lauriya,5,27.4,7.0,True
75,Election 1,1,Pradeep Singh,21775,Janata Dal (United),Nautan,6,22.7,0.0,True
105,Election 1,1,Renu Devi,48559,Bharatiya Janta Party,Bettiah,8,40.6,9.6,True


In [48]:
analysis.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 523 entries, 0 to 12289
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Election Year    523 non-null    object 
 1   Position         523 non-null    int64  
 2   Name             523 non-null    object 
 3   Votes            523 non-null    int64  
 4   Party            523 non-null    object 
 5   AC name          523 non-null    object 
 6   AC No            523 non-null    int64  
 7   Votes %          523 non-null    float64
 8   per_diff         523 non-null    float64
 9   isclose_contest  523 non-null    object 
dtypes: float64(2), int64(3), object(5)
memory usage: 44.9+ KB


In [49]:
analysis=analysis.pivot_table('per_diff','AC No','Election Year')
analysis

Election Year,Election 1,Election 2,Election 3,Election 4
AC No,,,,
1,9.6,2.5,NaN,NaN
3,0.3,0.9,NaN,NaN
4,NaN,5.3,NaN,4.9
5,7.0,7.2,9.4,NaN
6,0.0,2.1,NaN,9.6
...,...,...,...,...
239,7.9,1.7,4.5,NaN
240,2.2,9.3,NaN,5.8
241,NaN,NaN,NaN,5.2


In [50]:
def isbool(data):
    if (data[0]< 10) & (data[1]< 10) & (data[2]< 10) & (data[3]< 10):
        return 'True'
    else:
        return 'False'
analysis['bool']=analysis.apply(isbool,axis=1)

In [51]:
analysis=analysis[analysis['bool']!='False']
analysis

Election Year,Election 1,Election 2,Election 3,Election 4,bool
AC No,,,,,
9,8.3,5.6,7.0,1.8,True
10,2.7,8.7,8.9,1.9,True
25,5.8,1.1,3.7,2.4,True
28,1.4,4.5,4.4,8.9,True
31,1.9,4.3,6.0,2.6,True
36,9.6,3.2,0.5,4.3,True
37,8.9,2.3,2.3,4.0,True
52,2.0,2.7,3.4,8.8,True
53,3.8,3.0,5.4,4.5,True


In [52]:
analysis=analysis.reset_index()
analysis.columns

Index(['AC No', 'Election 1', 'Election 2', 'Election 3', 'Election 4',
       'bool'],
      dtype='object', name='Election Year')

In [53]:
analysis.head()
analysis=analysis[['AC No','bool']]

In [54]:
data_ops.head()

,Election Year,Position,Name,Votes,Party,AC name,AC No,Votes %
0,Election 3,1,Rajesh Singh,42289,Janata Dal (United),Valmiki Nagar,1,29.4
1,Election 3,2,Mukesh Kumar Kushwaha,27618,Rashtriya Janata Dal,Valmiki Nagar,1,19.2
2,Election 3,3,Dheerendra Pratap Singh,20886,Bahujan Samaj Party,Valmiki Nagar,1,14.5
3,Election 3,4,Irshad Hussain,17747,Indian National Congress,Valmiki Nagar,1,12.4
4,Election 3,5,Deep Narayan Mahato,14047,Independent,Valmiki Nagar,1,9.8


In [55]:
##to print relevant information need to drop some columns from data_ops 
essential_column_df=data_ops[['AC No','AC name']]

In [56]:
closed_contest_constituency=analysis.merge(essential_column_df,how='left',left_on='AC No',right_on='AC No')
closed_contest_constituency=closed_contest_constituency.dropna()
closed_contest_constituency=closed_contest_constituency.rename(columns={'AC No':'Constituency Code','AC name':'Constituency Name'})

In [57]:
df=closed_contest_constituency['Constituency Name']+'-'+closed_contest_constituency['Constituency Code'].map(str)

In [58]:
closed_cotest_constituency_list=pd.Series(df.unique().tolist())

In [59]:
closed_cotest_constituency_list.to_excel(Writer,sheet_name='closed contest constituency',index=False)

## 8.>  Find the growing seats. For the top 8 parties at the state-level in the 2005-Feb Election, Determine the seats in which these parties have continuously increased their votes in the next 3 elections.


In [60]:
top_8_2005_feb=top_8_parties[top_8_parties['Election Year']=='Election 1']
top_8_2005_feb

,Election Year,Party,Seat Count
0,Election 1,Rashtriya Janata Dal,75
1,Election 1,Janata Dal (United),55
2,Election 1,Bharatiya Janta Party,37
3,Election 1,Lok Jan Shakti Party,29
4,Election 1,Independent,15
5,Election 1,Indian National Congress,10
6,Election 1,Communist Party Of India (MARXIST-LENINIST) (L...,7
7,Election 1,Samajwadi Party,4


In [61]:
data.head()

,Election Year,Position,Name,Votes,Votes %,Party,AC name,AC No,Primary Key,Primary Key min
0,Election 3,1,Rajesh Singh,42289,29.40%,Janata Dal (United),Valmiki Nagar,1,1_42289,Election 3_42289_1
1,Election 3,2,Mukesh Kumar Kushwaha,27618,19.20%,Rashtriya Janata Dal,Valmiki Nagar,1,1_27618,Election 3_27618_1
2,Election 3,3,Dheerendra Pratap Singh,20886,14.50%,Bahujan Samaj Party,Valmiki Nagar,1,1_20886,Election 3_20886_1
3,Election 3,4,Irshad Hussain,17747,12.40%,Indian National Congress,Valmiki Nagar,1,1_17747,Election 3_17747_1
4,Election 3,5,Deep Narayan Mahato,14047,9.80%,Independent,Valmiki Nagar,1,1_14047,Election 3_14047_1


In [62]:
# conctructing primary key for op 8 paries in 2005 feb election('Election 1')
temp_data_feb_2005=data.merge(top_8_2005_feb,how='left',left_on='Party',right_on='Party')
temp_data_feb_2005=temp_data_feb_2005.dropna()
temp_data_feb_2005=temp_data_feb_2005.drop(['Election Year_y','Primary Key','Primary Key min',"Seat Count"],axis=1)
temp_data_feb_2005=temp_data_feb_2005.rename(columns={'Election Year_x':'Election Year'})

In [63]:
pivoted_data=temp_data_feb_2005.pivot_table(values='Votes',index=['AC No','Party'],columns='Election Year').dropna()
pivoted_data=pivoted_data.reset_index()

In [64]:
pivoted_data.columns

Index(['AC No', 'Party', 'Election 1', 'Election 2', 'Election 3',
       'Election 4'],
      dtype='object', name='Election Year')

In [65]:
pivoted_data.head()

Election Year,AC No,Party,Election 1,Election 2,Election 3,Election 4
0,1,Independent,5966.000000,707.25,3489.142857,16585.000000
1,2,Independent,1600.000000,1614.00,1289.166667,1803.666667
2,3,Bharatiya Janta Party,20961.000000,39147.00,45022.000000,41151.000000
3,3,Independent,707.583333,1512.50,5915.200000,7666.833333
4,3,Samajwadi Party,2735.000000,1515.00,1300.000000,1359.000000


In [66]:
df=pivoted_data.drop(['AC No','Party'],axis=1)
df1=df.T.diff().T.dropna(axis=1)
df1=df1.rename(columns={'Election 2':'Election2-Election1','Election 3':'Election3-Election2','Election 4':'Election4-Election3'})
pivoted_data_temp=pd.concat([pivoted_data,df1],axis=1)
pivoted_data_temp.reset_index(drop=True)

Election Year,AC No,Party,Election 1,Election 2,Election 3,Election 4,Election2-Election1,Election3-Election2,Election4-Election3
0,1,Independent,5966.000000,707.250000,3489.142857,16585.000000,-5258.750000,2781.892857,13095.857143
1,2,Independent,1600.000000,1614.000000,1289.166667,1803.666667,14.000000,-324.833333,514.500000
2,3,Bharatiya Janta Party,20961.000000,39147.000000,45022.000000,41151.000000,18186.000000,5875.000000,-3871.000000
3,3,Independent,707.583333,1512.500000,5915.200000,7666.833333,804.916667,4402.700000,1751.633333
4,3,Samajwadi Party,2735.000000,1515.000000,1300.000000,1359.000000,-1220.000000,-215.000000,59.000000
...,...,...,...,...,...,...,...,...,...
404,241,Independent,8809.375000,8443.166667,2290.250000,2685.333333,-366.208333,-6152.916667,395.083333
405,241,Rashtriya Janata Dal,9171.000000,26583.000000,35663.000000,66577.000000,17412.000000,9080.000000,30914.000000
406,242,Independent,6333.444444,4865.000000,2108.600000,5829.000000,-1468.444444,-2756.400000,3720.400000
407,242,Janata Dal (United),216.000000,3478.000000,48080.000000,43451.000000,3262.000000,44602.000000,-4629.000000


In [67]:
pivoted_data_temp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 409 entries, 0 to 408
Data columns (total 9 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   AC No                409 non-null    int64  
 1   Party                409 non-null    object 
 2   Election 1           409 non-null    float64
 3   Election 2           409 non-null    float64
 4   Election 3           409 non-null    float64
 5   Election 4           409 non-null    float64
 6   Election2-Election1  409 non-null    float64
 7   Election3-Election2  409 non-null    float64
 8   Election4-Election3  409 non-null    float64
dtypes: float64(7), int64(1), object(1)
memory usage: 28.9+ KB


In [68]:
def increasing_vote_fun(data):
    if data[-1]>0 and data[-2]>0 and data[-3]>0:
        return 'Increasing Vote'
    elif data[-1]<0 and data[-2]<0 and data[-3]<0:
        return 'Declining Vote'
    else:
        return 'Not Important'
pivoted_data_temp['Increasing_Declining']=pivoted_data_temp.apply(increasing_vote_fun,axis=1)

In [69]:
increasing_vote_constituency=pivoted_data_temp[pivoted_data_temp['Increasing_Declining']=='Increasing Vote']
increasing_vote_constituency.reset_index(drop=True)

Election Year,AC No,Party,Election 1,Election 2,Election 3,Election 4,Election2-Election1,Election3-Election2,Election4-Election3,Increasing_Declining
0,3,Independent,707.583333,1512.500000,5915.2,7666.833333,804.916667,4402.700000,1751.633333,Increasing Vote
1,6,Janata Dal (United),21775.000000,28089.000000,40894.0,52362.000000,6314.000000,12805.000000,11468.000000,Increasing Vote
2,7,Bharatiya Janta Party,27642.000000,42960.000000,44835.0,61304.000000,15318.000000,1875.000000,16469.000000,Increasing Vote
3,9,Janata Dal (United),29252.000000,35497.000000,40450.0,69870.000000,6245.000000,4953.000000,29420.000000,Increasing Vote
4,10,Bharatiya Janta Party,23640.000000,38448.000000,48686.0,64731.000000,14808.000000,10238.000000,16045.000000,Increasing Vote
...,...,...,...,...,...,...,...,...,...,...
86,233,Rashtriya Janata Dal,23183.000000,24023.000000,35023.0,60687.000000,840.000000,11000.000000,25664.000000,Increasing Vote
87,234,Bharatiya Janta Party,16422.000000,26197.000000,38893.0,67348.000000,9775.000000,12696.000000,28455.000000,Increasing Vote
88,235,Bharatiya Janta Party,12126.000000,25383.000000,51020.0,65934.000000,13257.000000,25637.000000,14914.000000,Increasing Vote
89,238,Independent,505.800000,611.285714,3962.0,9793.600000,105.485714,3350.714286,5831.600000,Increasing Vote


In [70]:
increasing_vote_constituency['Party'].value_counts().sort_values(ascending=False).to_excel(Writer,sheet_name='Growing Seat Party')

## 9.> Find the declining seats. Repeat previous exercise  for the opposite growth.

In [71]:
Declining_constituency_vote=pivoted_data_temp[pivoted_data_temp['Increasing_Declining']=='Declining Vote']
Declining_constituency_vote

Election Year,AC No,Party,Election 1,Election 2,Election 3,Election 4,Election2-Election1,Election3-Election2,Election4-Election3,Increasing_Declining
40,18,Samajwadi Party,2774.000000,934.0,912.000000,573.000000,-1840.000000,-22.000000,-339.000000,Declining Vote
109,61,Independent,3821.400000,3244.0,2162.222222,2067.583333,-577.400000,-1081.777778,-94.638889,Declining Vote
165,94,Independent,2197.500000,2140.0,2096.714286,688.200000,-57.500000,-43.285714,-1408.514286,Declining Vote
207,123,Independent,6067.125000,5237.5,2000.285714,1300.571429,-829.625000,-3237.214286,-699.714286,Declining Vote
211,124,Samajwadi Party,6630.000000,2391.0,785.000000,480.000000,-4239.000000,-1606.000000,-305.000000,Declining Vote
218,128,Independent,3214.000000,2819.0,1970.000000,1339.250000,-395.000000,-849.000000,-630.750000,Declining Vote
249,147,Independent,4304.714286,3533.2,3454.000000,1506.500000,-771.514286,-79.200000,-1947.500000,Declining Vote
302,183,Independent,1568.750000,1458.8,584.600000,529.181818,-109.950000,-874.200000,-55.418182,Declining Vote
325,197,Independent,3482.666667,2661.0,2261.000000,1923.000000,-821.666667,-400.000000,-338.000000,Declining Vote
365,216,Independent,5188.666667,2855.0,2466.500000,2033.750000,-2333.666667,-388.500000,-432.750000,Declining Vote


In [72]:
Declining_constituency_vote['Party'].value_counts().sort_values(ascending=False).to_excel(Writer,sheet_name='Declining Seat Party')

## 10.> Determine the state-winner-match constituencies. Find the seats which have always elected the party that has gone on to win the state election (most votes secured at state). So if the RJD won at the state level in 2010, then seats which voted an RJD candidate as winner are considered in this category. This criteria for a seat has to be matched for each election

In [73]:
winner_list

,Election Year,Position,Name,Votes,Votes %,Party,AC name,AC No_x,Primary Key
0,Election 3,1,Rajesh Singh,42289,29.40%,Janata Dal (United),Valmiki Nagar,1,1_42289
1,Election 3,1,Bhagirathi Devi,51993,41.50%,Bharatiya Janta Party,Ramnagar,2,2_51993
2,Election 3,1,Satish Chandra Dubey,45022,38.10%,Bharatiya Janta Party,Narkatiaganj,3,3_45022
3,Election 3,1,Prabhat Ranjan Singh,67510,50.40%,Janata Dal (United),Bagaha,4,4_67510
4,Election 3,1,Vinay Bihari,38381,33.40%,Independent,Lauriya,5,5_38381
...,...,...,...,...,...,...,...,...,...
961,Election 1,1,Purnima Yadav,53995,42.40%,Independent,Warsaliganj,239,239_53995
962,Election 1,1,Nand Kishore Choudhary,34272,38.70%,Rashtriya Janata Dal,Sikandra (SC),240,240_34272
963,Election 1,1,Kaushal Yadav,65224,63.50%,Independent,Jamui,241,241_65224
964,Election 1,1,Aruna Devi,60999,50.50%,Lok Jan Shakti Party,Jhajha,242,242_60999


In [74]:
winner_list['winner_match_key']=winner_list['Election Year']+'_'+winner_list['Party']
winner_every_election['winner_match_key']=winner_every_election['Election Year']+'_'+winner_every_election['Party']
state_winner_match=winner_list.merge(winner_every_election,how='left',left_on='winner_match_key',right_on='winner_match_key')
state_winner_match=state_winner_match.dropna()
state_winner_match=state_winner_match.drop(['Election Year_y','Party_y','Primary Key'],axis=1)
state_winner_match=state_winner_match.rename(columns={'Election Year_x':'Election Year','Party_x':'Party','AC No_x':'AC  No'})
state_winner_match.to_excel('1.xlsx',index=False)

In [75]:
state_winner_match=state_winner_match.pivot_table(index='AC name',columns='Election Year',values='winner_match_key',aggfunc='count').reset_index()
state_winner_match=state_winner_match.fillna(value=0,axis=0)

In [76]:
state_winner_match.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 230 entries, 0 to 229
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   AC name     230 non-null    object 
 1   Election 1  230 non-null    float64
 2   Election 2  230 non-null    float64
 3   Election 3  230 non-null    float64
 4   Election 4  230 non-null    float64
dtypes: float64(4), object(1)
memory usage: 9.1+ KB


In [77]:
state_winner_match.head()

Election Year,AC name,Election 1,Election 2,Election 3,Election 4
0,Alamnagar,1.0,0.0,1.0,0.0
1,Alauli,0.0,1.0,1.0,1.0
2,Alinagar,0.0,0.0,0.0,1.0
3,Amarpur,0.0,0.0,1.0,0.0
4,Amnour,0.0,1.0,1.0,0.0


In [78]:
def winner_match_election_wise(data):
    if (data[-1]>0) and (data[-2]>0) and (data[-3]>0) and (data[-4]>0):
        return 'four Times State Winner Matches with constituency winner'
    elif ((data[-1]>0) and (data[-2]>0) and (data[-3]>0)) or ((data[-1]>0) and (data[-2]>0) and (data[-4]>0)) or ((data[-1]>0) and (data[-3]>0) and (data[-4]>0))\
         or ((data[-3]>0) and (data[-2]>0) and (data[-4]>0)):
        return 'Three Times State Winner Matches with constituency winner'
    elif ((data[-1]>0) and (data[-2]>0)) or (data[-1]>0) and (data[-3]>0) or (data[-1]>0) and (data[-4]>0)\
          or (data[-2]>0) and (data[-3]>0) or (data[-2]>0) and (data[-4]>0) or (data[-3]>0) and (data[-4]>0):
        return 'Two Times State Winner Matches with constituency winner'
    elif(data[-1]>0) or (data[-2]>0) or (data[-3]>0) or (data[-4]>0):
            return 'One Times State Winner Matches with constituency winner'
state_winner_match['Election Wise Matching']=state_winner_match.apply(winner_match_election_wise,axis=1)

In [79]:
state_winner_match.to_excel(Writer,sheet_name='State winner cons winner match',index=False)

In [80]:
Writer.save()